In [1]:
!pip freeze | grep scikit-learn

'grep' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import pickle
import pandas as pd

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [4]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [5]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet')

In [6]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [7]:
y_pred.std()

5.28140357655334

In [9]:
year = 2022
month = 2
df['ride_id'] = f'{year:04d}/{month:02d}_'+df.index.astype('str')


In [12]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['pred'] = y_pred

In [13]:
df_result.to_parquet(
    './output.parquet',
    engine = 'pyarrow',
    compression = None,
    index = False
)

In [14]:
import os

In [16]:
!dir

 Volume in drive D is ? ??
 Volume Serial Number is 3099-9011

 Directory of d:\Code\GitHubLawn\MLOpsZoomcamp\hw4

2023-06-18  ?? 10:12    <DIR>          .
2023-06-18  ?? 10:12    <DIR>          ..
2023-06-18  ?? 10:03                77 Dockerfile
2023-06-18  ?? 10:03            17,369 model.bin
2023-06-18  ?? 10:12        59,994,813 output.parquet
2023-06-18  ?? 10:00             2,198 starter.ipynb
               4 File(s)     60,014,457 bytes
               2 Dir(s)  992,112,209,920 bytes free


In [1]:
!jupyter nbconvert --to python starter.ipynb

[NbConvertApp] Converting notebook starter.ipynb to python
[NbConvertApp] Writing 884 bytes to starter.py
